### 子类化内置类型

In [1]:
# 一个不允许重复值的字典
class DistinctError(ValueError):
    """如果向distinctdict添加重复值，则引发这个错误"""
    ...

class DistinctDict(dict):
    """不接受重复值的字典"""
    def __setitem__(self, key, value):
        if value in self.values():
            if (
                    (key in self and self[key] != value) or key not in self
            ):
                raise DistinctError("this value already exists for different key")

        super().__setitem__(key, value)

my = DistinctDict()
my['key'] = 'value'
my['other_key'] = 'value'

DistinctError: this value already exists for different key

In [2]:
my['other_key'] = 'value2'
my

{'key': 'value', 'other_key': 'value2'}

### 访问超类中的方法

In [3]:
# 访问超类中的方法
# 旧的写法
class Mama:
    def says(self):
        print('do your homework')


class Sister(Mama):
    def says(self):
        Mama.says(self)
        print('and clean your bedroom')

Sister().says()

do your homework
and clean your bedroom


In [4]:
# super方法访问超类
class Sister(Mama):
    def says(self):
        super().says() # 简化形式可以在方法内部使用
        print('and clean your bedroom')

Sister().says()

do your homework
and clean your bedroom


In [5]:
# super不在方法内部使用时
anita = Sister()
super(anita.__class__, anita).says()

do your homework


In [13]:
# 如果super只提供了一个参数，那么它将返回一个未绑定（unbound）类型
# 这一点在与classmethod一起使用时特别有用
class Pizza:
    def __init__(self, toppings):
        self.toppings = toppings

    def __repr__(self):
        return 'Pizza with ' + ' and '.join(self.toppings)

    @classmethod
    def recommend(cls):
        """推荐任意馅料（toppings）的某种披萨"""
        return cls(['spam', 'ham', 'eggs'])


class VikingPizza(Pizza):
    @classmethod
    def recommend(cls):
        """推荐与super相同的内容，但多加了午餐肉（spam）"""
        recommended = super().recommend()
        recommended.toppings += ['spam'] * 5
        return recommended

print(Pizza.recommend())
print(VikingPizza.recommend())

Pizza with spam and ham and eggs
Pizza with spam and ham and eggs and spam and spam and spam and spam and spam


In [20]:
# 错误案例 - 混用super与显式类调用
class A:
    def __init__(self):
        print('A', end=' ')
        super().__init__()


class B:
    def __init__(self):
        print('B', end=' ')
        super().__init__()


class C(A, B):
    def __init__(self):
        print('C', end=' ')
        A.__init__(self)
        B.__init__(self)


# 类的继承顺序
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

In [21]:
# C的实例调用了A.__init__(self) -> super(A, self).__init__() —> B.__init__()方法
# 因此会打印2次B
C()

C A B B 

In [43]:
# 初始化过程中的参数传递
class CommonBase:
    def __init__(self):
        print('CommonBase')
        super().__init__()


class Base1(CommonBase):
    def __init__(self):
        print('Base1')
        super().__init__()


class Base2(CommonBase):
    def __init__(self, arg):
        print('Base2')
        super().__init__()


class MyClass(Base1, Base2):
    def __init__(self, arg):
        print('my base')
        super().__init__(arg)

MyClass(10)

my base


TypeError: Base1.__init__() takes 1 positional argument but 2 were given

In [44]:
MyClass.__mro__

(__main__.MyClass, __main__.Base1, __main__.Base2, __main__.CommonBase, object)

### MRO

In [16]:
# MRO：方法解析顺序
class CommonBase:
    def method(self):
        print('CommonBase')


class Base1(CommonBase):
    pass


class Base2(CommonBase):
    def method(self):
        print('Base2')


class MyClass(Base1, Base2):
    pass


# 调用类中方法的顺序
MyClass.__mro__

(__main__.MyClass, __main__.Base1, __main__.Base2, __main__.CommonBase, object)

In [17]:
MyClass().method()

Base2


### 高级属性访问模式

In [23]:
# 名称修饰：以__开始的变量是类的私有属性
# 直接访问私有属性会报错，该属性会被解释器重命名
class MyClass:
    __secret_value = 1


instance_of = MyClass()
instance_of.__secret_value

AttributeError: 'MyClass' object has no attribute '__secret_value'

In [24]:
dir(MyClass)

['_MyClass__secret_value',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [25]:
instance_of._MyClass__secret_value

1

In [26]:
# 描述符允许你自定义在引用一个对象的属性时应该完成的事
# 描述符是Python中复杂属性访问的基础
# 它基于3个特殊方法，这3个方法组成了描述符协议：
# 1.__set__(self, instance, value)：在设置属性时将调用这一方法
# 2.__get__(self, instance, owner)：在读取属性时将调用这一方法
# 3.__delete__(self, obj)：对属性调用del时将调用这一方法
# 实现了__get__()和__set__()的描述符被称为数据描述符
class RevealAccess:
    """一个数据描述符，正常设定值并返回值，同时打印出记录访问的信息"""
    def __init__(self, initval=None, name='var'):
        self.value = initval
        self.name = name

    def __get__(self, instance, owner):
        print('Retrieving', self.name)
        return self.value

    def __set__(self, instance, value):
        print('Updating', self.name)
        self.value = value


class MyClass:
    x = RevealAccess(10, 'var x')
    y = 5

m = MyClass()
m.x

Retrieving var x


10

In [27]:
m.x = 20

Updating var x


In [28]:
m.x

Retrieving var x


20

In [29]:
m.y

5

In [30]:
# 描述符的一个示例：延迟求值属性
class InitOnAccess:
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        self._initialized = None

    def __get__(self, instance, owner):
        if self._initialized is None:
            print('initialized!')
            self._initialized = self.klass(*self.args, **self.kwargs)
        else:
            print('cached')
        return self._initialized


class MyClass:
    lazily_initialized = InitOnAccess(list, 'argument')

m = MyClass
m.lazily_initialized

initialized!


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']

In [31]:
m.lazily_initialized

cached


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']

### property

In [1]:
# property：一个内置的描述符，它知道如何将一个属性链接到一组方法上
# property接受4个参数：fget、fset、fdel和doc
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2

    def _width_get(self):
        return self.x2 - self.x1

    def _width_set(self, value):
        self.x2 = self.x1 + value

    def _height_get(self):
        return self.y2 - self.y1

    def _height_set(self, value):
        self.y2 = self.y1 + value

    width = property(_width_get, _width_set, doc='rectangle width measured from left')
    height = property(_height_get, _height_set, doc='rectangle height measured from top')

    def __repr__(self):
        return f'{self.__class__.__name__}({self.x1}, {self.y1}, {self.x2}, {self.y2})'


# 实例化Rectangle对象
rec = Rectangle(10, 10, 25, 34)

# 获取对象实例的属性值时，property依据传入的fget进行处理
rec.width, rec.height

(15, 24)

In [2]:
# 更改对象实例的属性值时，property依据传入的fset进行处理
rec.width = 100
rec

Rectangle(10, 10, 110, 34)

In [3]:
rec.height = 100
rec

Rectangle(10, 10, 110, 110)

In [4]:
help(rec)

Help on Rectangle in module __main__ object:

class Rectangle(builtins.object)
 |  Rectangle(x1, y1, x2, y2)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x1, y1, x2, y2)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  height
 |      rectangle height measured from top
 |  
 |  width
 |      rectangle width measured from left



In [5]:
# 但在使用类的继承事必须小心处理，所创建的属性时利用当前类的方法实时创建的，不会使用派生类中覆写的方法
class MetricRectangle(Rectangle):
    def _width_get(self):
        return f'{self.x2 - self.x1} meters'


MetricRectangle(0, 0, 100, 100).width

100

In [6]:
# 为了解决上述问题，需要在派生类中覆写整个property
class MetricRectangle(Rectangle):
    def _width_get(self):
        return f'{self.x2 - self.x1} meters'
    width = property(_width_get, Rectangle.width.fset)


MetricRectangle(0, 0, 100, 100).width

'100 meters'

In [12]:
# 上述方法存在维护性问题：如果更改父类，却忘记修改property调用的话，就会产生问题
# 因此推荐在派生类中覆写所有的property方法，而不是依赖父类实现
# 基于以上原因，property的最佳语法是将其作为装饰器
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2

    @property
    def width(self):
        """rectangle width measured from left"""
        return self.x2 - self.x1

    @width.setter
    def width(self, value):
        self.x2 = self.x1 + value

    @property
    def height(self):
        """rectangle width measured from top"""
        return self.y2 - self.y1

    @height.setter
    def height(self, value):
        self.y2 = self.y1 + value

    def __repr__(self):
        return f'{self.__class__.__name__}({self.x1}, {self.y1}, {self.x2}, {self.y2})'


rec = Rectangle(10, 10, 25, 34)
rec.width

15

In [13]:
rec.width = 100
rec

Rectangle(10, 10, 110, 34)

In [14]:
help(Rectangle)

Help on class Rectangle in module __main__:

class Rectangle(builtins.object)
 |  Rectangle(x1, y1, x2, y2)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x1, y1, x2, y2)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  height
 |      rectangle width measured from top
 |  
 |  width
 |      rectangle width measured from left



### 槽（slots）

In [16]:
# __slots__可以为类设置一个静态属性列表，并在类的每个实例中跳过__dict__字典的创建过程
# 它可以为属性很少的类节约内存空间，此外，它还有助于设计签名需要被冻结的类
# 例如，限制一个类的语言动态特性
class Frozen:
    __slots__ = ['ice', 'cream']


'__dict__' in dir(Frozen)

False

In [17]:
'ice' in dir(Frozen)

True

In [18]:
frozen = Frozen()
frozen.ice = True
frozen.cream = None
frozen.icy = True

AttributeError: 'Frozen' object has no attribute 'icy'

### 元编程

In [29]:
# 元编程：一种编写计算机程序的技术，这些程序可以将自己看做数据
# 因此，你可以在运行时对它进行内省、生成或修改
# python元编程方法：装饰器、类的特殊方法、元类
# 类装饰器：与装饰器类似，但返回值是一个类，而不是函数对象
def short_repr(cls):
    """一个修改任意类repr方法的类装饰器"""
    cls.__repr__ = lambda self: super(cls, self).__repr__()[:8]
    return cls

@short_repr
class ClassWithLongName:
    pass


ClassWithLongName()

<__main_

In [30]:
# 编写函数装饰器的其他内容也适用于类装饰器
# 最重要的，它可以使用闭包（在函数内部引用外部作用域的变量）
def parametrized_short_repr(max_width=8):
    """缩短表示的参数化装饰器"""
    def parametrized(cls):
        """内部包装函数，是实际的装饰器"""
        class ShortlyRepresented(cls):
            """提供装饰器行为的子类"""
            def __repr__(self):
                return super().__repr__()[:max_width]

        return  ShortlyRepresented

    return  parametrized


# 在类装饰器中使用闭包的主要缺点是，生成的对象不在是被装饰的类的实例，而是在装饰器中动态创建的子类的实例
# 这会影响类的__name__和__doc__等属性
@parametrized_short_repr(10)
class ClassWithLittleBitLongerLongName:
    pass


ClassWithLittleBitLongerLongName().__class__

__main__.parametrized_short_repr.<locals>.parametrized.<locals>.ShortlyRepresented

In [31]:
ClassWithLittleBitLongerLongName().__doc__

'提供装饰器行为的子类'

In [32]:
# 使用__new__()方法覆写实例创建过程
# __new__()是一种负责创建类实例的静态方法，其调用优先于__init__()初始化方法
# 通常，覆写__new__()的实现将会使用合适的参数调用超类的super().__new__()，并在返回前修改实例
class InstanceCountingClass:
    instances_created = 0
    def __new__(cls, *args, **kwargs):
        print("__new__() called with", cls, args, kwargs)
        instance = super().__new__(cls)
        instance.number = cls.instances_created
        cls.instances_created += 1
        return instance

    def __init__(self, attr):
        print('__init__() called with', self, attr)
        self.attr = attr


instance1 = InstanceCountingClass('abc')

__new__() called with <class '__main__.InstanceCountingClass'> ('abc',) {}
__init__() called with <__main__.InstanceCountingClass object at 0x000001FF420C40D0> abc


In [33]:
instance2 = InstanceCountingClass('xyz')

__new__() called with <class '__main__.InstanceCountingClass'> ('xyz',) {}
__init__() called with <__main__.InstanceCountingClass object at 0x000001FF420C4130> xyz


In [34]:
instance1.number, instance1.instances_created

(0, 2)

In [35]:
instance2.number, instance2.instances_created

(1, 2)

In [38]:
# 通常，__new__()方法应该返回该类的一个实例，但也可以返回其他类的实例
# 当返回了其他类的实例事，那么将跳过对__init__()方法的调用
class NonZero(int):
    """一个非零的整数子类"""
    def __new__(cls, value):
        return super().__new__(cls, value) if value != 0 else None

    def __init__(self, x):
        print('__init__() called')
        super().__init__()


type(NonZero(-12))

__init__() called


__main__.NonZero

In [39]:
type(NonZero(0))

NoneType

In [40]:
# 元类：定义其他类（型）的一种类（型），python中所有类定义的基类都是内置的type类
# 我们可以将某个类的元类替换为我们自定义的类
# 通常，新的元类仍然是type类的子类
# 调用内置的type()类可以作为class语句的动态等效
# 给定名称，基类和包含的属性的映射，它会创建一个新的类对象
def method(self):
    return 1

klass = type('MyClass', (object,), {'method': method})
instance = klass()
instance.method()

1

In [ ]:
# 用class语句创建的每个类都会隐式地使用type作为其元类
# 可以通过向class语句提供metaclass关键字参数来改变这一默认行为
# metaclass参数的值通常是另一个类对象，但它可以是任意可调用对象，只要接受与type类相同的参数并返回另一个类对象即可
# 调用签名为type(name, bases, namespace)
# 元类的常用模板：
class MataClass(type):
    def __new__(mcs, name, bases, namespace):
        """
        复杂类对象的实际创建，其创建方式与普通类相同。
        第一个位置的参数时一个元类对象，在本例中就是MetaClass

        注意：mcs是这一参数常用的命名约定

        :param name: 这是将保存在__name__属性中的类名称
        :param bases: 这是父类的列表，将成为__bases__属性，并用于构建新创建的类的MRO
        :param namespace: 这是包含类主体定义的命名空间（映射），将成为__dict__属性
        """
        return super().__new__(mcs, name, bases, namespace)

    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        """
        创建一个空的命名空间对象。

        注意：它不接受namespace作为参数，因为在它调用前命名空间不存在。

        :return: 默认返回一个空的dict但可以覆写并使其返回其他任何映射类型
        """
        return super().__prepare__(name, bases, **kwargs)

    def __init__(cls, name, bases, namespace, **kwargs):
        """
        这在元类的实现中并不常见，但其含义与普通类中的含义相同。一旦__new__()创建完成，
        它可以执行其他类对象的初始化过程。现在第一个位置参数的命名约定为cls，说明它已经
        是一个创建好的类对象（元类的实例），而不是一个元类对象。__init__()被调用时，类
        的构建已经完成，所以这一方法可以做的事情比__new__()要少。实现这样的方法非常类似
        使用类装饰器，但主要区别在于，每个子类都会调用__init__()，而类装饰器则不会被子
        类调用。
        """
        super().__init__(name, bases, namespace)

    def __call__(cls, *args, **kwargs):
        """
        当调用元类实例时会调用这一方法。元类实例一个类对象，在创建类的新的实例时会调用它。
        这一方法可以用于覆写类实例创建和初始化的默认方式。
        """
        return super().__call__(*args, **kwargs)

In [41]:
# 元类 - 一个具体的例子
class RevealingMeta(type):
    def __new__(mcs, name, bases, namespace):
        print(mcs, '__new__ called')
        return super().__new__(mcs, name, bases, namespace)

    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print(mcs, '__prepare__ called')
        return super().__prepare__(name, bases, **kwargs)

    def __init__(cls, name, bases, namespace, **kwargs):
        print(cls, '__init__ called')
        super().__init__(name, bases, namespace)

    def __call__(cls, *args, **kwargs):
        print(cls, '__call__ called')
        return super().__call__(*args, **kwargs)


# 利用RevealingMeta作为元类，创建新的类定义
class RevealingClass(metaclass=RevealingMeta):
    def __new__(cls, *args, **kwargs):
        print(cls, '__new__ called')
        return super().__new__(cls)

    def __init__(self):
        print(self, '__init__ called')
        super().__init__()

<class '__main__.RevealingMeta'> __prepare__ called
<class '__main__.RevealingMeta'> __new__ called
<class '__main__.RevealingClass'> __init__ called


In [42]:
instance = RevealingClass()

<class '__main__.RevealingClass'> __call__ called
<class '__main__.RevealingClass'> __new__ called
<__main__.RevealingClass object at 0x000001FF422C6980> __init__ called
